In [162]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Solving Linear System
## Finding Lm Ln Polynomials 

In [163]:
def legendre_polynomial(n):
    y = sp.symbols('y')
    if n == 0:
        return sp.Lambda(y, 1)
    elif n == 1:
        return sp.Lambda(y, y)
    else:
        P_n_minus_1 = legendre_polynomial(n-1)
        P_n_minus_2 = legendre_polynomial(n-2)
        return sp.Lambda(y, ((2*n - 1) * y * P_n_minus_1(y) - (n - 1) * P_n_minus_2(y)) / n)

# Example usage:
n = 2
Ln = legendre_polynomial(n)
Ln


Lambda(y, 3*y**2/2 - 1/2)

## Computing Anm using GL Quadrature --> finding A 

In [164]:
n = [0]
m = [0]

def Anm(n, m):
    y = sp.symbols('y')
    Ln = legendre_polynomial(n)
    Lm = legendre_polynomial(m)
    return 0 * Ln(y) * Lm(y)
    #    return y * Ln(y) * Lm(y) for a non deterministic case

def Anm_gauleg(xi, ci, b , a, n2, n, m):
    sum = 0 
    Anm_expr = Anm(n,m)
    for i in range(n2):
        y_value = (0.5*(((b-a)*xi[i])+(b+a)))
        prod = ci[i]* Anm_expr.subs(sp.symbols('y'), y_value)
        sum = sum + prod 
    return sum 

def A_matrix(n,m):
    A_matrix = sp.zeros(len(n),len(m))
    n2 = math.floor((len(n)+len(m))/2) + 1
    xi = gl.gauleg(n2)[0] # satisfying dop of GL for p+1 is 2p + 1
    ci = gl.gauleg(n2)[1]
    for ni in n:
        for mi in m:
            A_matrix[ni ,mi] = Anm_gauleg(xi= xi, ci = ci, b = 1, a = -1, n2 = n2, n = ni, m = mi)

    return A_matrix



## Define A(x,y) 

In [165]:
xd =1/3

def a(x, y):
    return 2 + math.sin(x)


# def a0(x):
#     return 2 + math.sin(x)

def a0(x):
    if 0 <= x < xd: 
        return 1.0 
    elif xd <= x <= 1:
        return 2.0

# def a0(x):
#     if 0 <= x < 0.5:
#         return 1
#     elif 0.5 <= x <= 1:
#         return 2

# def a0(x): 
#     if 0<= x < 0.75:
#         return 1 
#     elif 0.75<= x <= 1:
#         return 2

def a1(x):
    return 0 *x



## Define (S1) matrix and (S0) matrix

Error Indexing 

nodal index = (0,1,2,3,...,n) 

element index =(0,1,...,n-1) 

finite element function index =  nodal index = (0,1,2,3...) since phi_i(x_i) = 1 for all i

eg: nodal index = (0,1,2)

element index = (0,1)

element i = (x_i, x_i+1)

In [166]:
def GL(xi, ci, x_right, x_left, n2, func):
	sum = 0 
	for i in range(n2):
		prod = ci[i]*func(0.5*(((x_right-x_left)*xi[i])+(x_right+x_left)))
		sum = sum + prod 
	return (x_right-x_left)/2 * (sum)

def piecewise_GL(integrand, x_left, x_right, n2):
    """
    Integrate 'integrand(x)' from x_left to x_right using Gauss-Legendre,
    but if [x_left, x_right] crosses x=1/3, split it into two parts:
      [x_left, 1/3]  and  [1/3, x_right]
    and sum the results.
    """
    discont = xd
    xi, ci = gl.gauleg(n2) 
    # If the entire interval is on one side of 1/3, do one normal G-L integral
    if x_right <= discont or x_left >= discont:
        return GL(xi =  xi, ci = ci, x_left=x_left, x_right=x_right, func=integrand, n2=n2)

    # Otherwise, we split at x=1/3
    # part1 = integral from x_left..1/3
    part1 = GL(xi =  xi, ci = ci, x_left=x_left, x_right=discont, func=integrand, n2=n2)
    # part2 = integral from 1/3..x_right
    part2 = GL(xi =  xi, ci = ci, x_left=discont, x_right=x_right, func=integrand, n2=n2)

    return part1 + part2

def dphi_i_on_element(i, k, xlist):
    """
    Return the (constant) slope of the i-th shape function on the k-th subinterval
    [ x_k, x_{k+1} ] in a 1D mesh with nodes x_0 < ... < x_N.
    """
    if i == k:
        # node i is the left endpoint of the subinterval => slope from 1 at x_k down to 0 at x_{k+1}
        dx = xlist[k+1] - xlist[k]
        return -1.0 / dx
    elif i == k+1:
        # node i is the right endpoint => slope from 0 at x_k up to 1 at x_{k+1}
        dx = xlist[k+1] - xlist[k]
        return +1.0 / dx
    else:
        return 0.0


def phiij(numofnodes, i, j, xlist, func):
    finalsum = 0
    for k in range(numofnodes-1):
        x_left = xlist[k]
        x_right = xlist[k+1]

        # Derivatives of shape functions phi_i, phi_j on element k
        d_phi_i = dphi_i_on_element(i, k, xlist)
        d_phi_j = dphi_i_on_element(j, k, xlist)
        def local_integrand(x_val):
            return func(x_val) * (d_phi_i * d_phi_j)

        # Now call piecewise_GL with local_integrand
        val = piecewise_GL(local_integrand, x_left, x_right, n2=5)
        finalsum += val
    return finalsum

def S1_ji(func, mesh):
    N = len(mesh)  # number of nodes
    S1_mat = sp.zeros(N, N)
    n2 = 5
    # Assemble diagonal entries S1[j,j]
    for j in range(N):
        diag_val = 0.0
        # Contribution from the left subinterval [mesh[j-1], mesh[j]]
        if j > 0:
            x_left = mesh[j-1]
            x_right = mesh[j]
            # On [mesh[j-1], mesh[j]], phi_j is the right (increasing) part.
            def integrand_left(x):
                # dphi_j/dx on [mesh[j-1], mesh[j]]:
                return func(x) * (dphi_i_on_element(j, j-1, mesh))**2
            diag_val += piecewise_GL(integrand_left, x_left, x_right, n2=n2)
            
        # Contribution from the right subinterval [mesh[j], mesh[j+1]]
        if j < N-1:
            x_left = mesh[j]
            x_right = mesh[j+1]
            # On [mesh[j], mesh[j+1]], phi_j is the left (decreasing) part.
            def integrand_right(x):
                # dphi_j/dx on [mesh[j], mesh[j+1]]:
                return func(x) * (dphi_i_on_element(j, j, mesh))**2
            diag_val += piecewise_GL(integrand_right, x_left, x_right, n2=n2)
            
        S1_mat[j, j] = diag_val

    for j in range(1, N):
        # Contribution for S1[j, j-1] (and by symmetry S1[j-1, j]) over [mesh[j-1], mesh[j]]
        x_left = mesh[j-1]
        x_right = mesh[j]
        def integrand_off(x):
            # For node j-1 on the element, phi_{j-1} is the left (decreasing) part,
            # and for node j, phi_j is the right (increasing) part.
            return func(x) * dphi_i_on_element(j-1, j-1, mesh) * dphi_i_on_element(j, j-1, mesh)
        val = piecewise_GL(integrand_off, x_left, x_right, n2=n2)
        S1_mat[j, j-1] = val
        S1_mat[j-1, j] = val  # Exploiting symmetry
        
    return S1_mat




In [167]:
def S0_ji(func, mesh):
    N = len(mesh)
    S0_mat = sp.zeros(N, N)
    n2 = 5
    # assembly of diagonal entries of S0 
    for j in range(N):
            diag_val = 0.0
            # Contribution from the left subinterval [mesh[j-1], mesh[j]]
            if j > 0:
                x_left = mesh[j-1]
                x_right = mesh[j]
                # On [mesh[j-1], mesh[j]], phi_j is the right (increasing) part.
                def integrand_left(x):
                    # dphi_j/dx on [mesh[j-1], mesh[j]]:
                    return func(x) * (dphi_i_on_element(j, j-1, mesh))**2
                diag_val += piecewise_GL(integrand_left, x_left, x_right, n2=n2)
                
            # Contribution from the right subinterval [mesh[j], mesh[j+1]]
            if j < N-1:
                x_left = mesh[j]
                x_right = mesh[j+1]
                # On [mesh[j], mesh[j+1]], phi_j is the left (decreasing) part.
                def integrand_right(x):
                    # dphi_j/dx on [mesh[j], mesh[j+1]]:
                    return func(x) * (dphi_i_on_element(j, j, mesh))**2
                diag_val += piecewise_GL(integrand_right, x_left, x_right, n2=n2)
                
            S0_mat[j, j] = diag_val

    for j in range(1, N):
        # Contribution for S1[j, j-1] (and by symmetry S1[j-1, j]) over [mesh[j-1], mesh[j]]
        x_left = mesh[j-1]
        x_right = mesh[j]
        def integrand_off(x):
            # For node j-1 on the element, phi_{j-1} is the left (decreasing) part,
            # and for node j, phi_j is the right (increasing) part.
            return func(x) * dphi_i_on_element(j-1, j-1, mesh) * dphi_i_on_element(j, j-1, mesh)
        val = piecewise_GL(integrand_off, x_left, x_right, n2=n2)
        S0_mat[j, j-1] = val
        S0_mat[j-1, j] = val  # Exploiting symmetry
        
    return S0_mat
        


## Define Kroneker Notation 


In [168]:
# n, m are integers, probably need to loop it for every possible n, m 

def delta_mn(m,n):
    if n == m:
        return 1 
    else:
        return 0 

def delta(n,m):
    delta = sp.zeros(len(n), len(m))
    for ni in n:
        for mi in m: 
            delta[ni,mi] = delta_mn(ni,mi)

    return delta 



## Kron Matrix A and S1 

In [169]:
def A_S1(matrixA, matrixS1):
    A_S1 = sp.kronecker_product(matrixA, matrixS1)
    return A_S1




## Define S0 * delta_mn


In [170]:
def S0_delta(matrixS0, matrixdelta):
    return sp.kronecker_product(matrixS0 , matrixdelta) 



## Define S(in)(jm)


In [171]:
def S_in_jm(matrixS1_Amn, matrixS0_delta):
    return matrixS0_delta + matrixS1_Amn



## Define Fi and F 

In [172]:
def GL(xi, ci, x_left, x_right, n2, func):
    """
    Performs a Gauss–Legendre integration on [x_left, x_right] of func(x).
    """
    ssum = 0.0
    for i in range(n2):
        # Map the Gauss–Legendre node from [-1,1] to [x_left,x_right]
        x_mapped = 0.5 * ((x_right - x_left)*xi[i] + (x_right + x_left))
        ssum += ci[i] * func(x_mapped)
    return 0.5 * (x_right - x_left) * ssum

def piecewise_GL(integrand, x_left, x_right, n2):
    """
    Integrate integrand(x) from x_left to x_right using Gauss-Legendre.
    If the interval crosses x=1/3, split it into [x_left, 1/3] and [1/3, x_right].
    """
    discont = xd
    xi, ci = gl.gauleg(n2)
    
    if x_right <= discont or x_left >= discont:
        return GL(xi=xi, ci=ci, x_left=x_left, x_right=x_right, func=integrand, n2=n2)
    else:
        part1 = GL(xi=xi, ci=ci, x_left=x_left, x_right=discont, func=integrand, n2=n2)
        part2 = GL(xi=xi, ci=ci, x_left=discont, x_right=x_right, func=integrand, n2=n2)
        return part1 + part2

def phi_i(i, x, mesh):
    """
    Standard 1D hat (finite element) function.
    This returns the value of the i-th hat function at x, given a mesh.
    """
    N = len(mesh) - 1  # number of elements
    # For node 0 (left boundary)
    if i == 0:
        if x < mesh[0] or x > mesh[1]:
            return 0.0
        else:
            return (mesh[1] - x) / (mesh[1] - mesh[0])
    # For node N (right boundary)
    elif i == N:
        if x < mesh[N-1] or x > mesh[N]:
            return 0.0
        else:
            return (x - mesh[N-1]) / (mesh[N] - mesh[N-1])
    # For interior nodes
    else:
        if x < mesh[i-1] or x > mesh[i+1]:
            return 0.0
        elif x <= mesh[i]:
            return (x - mesh[i-1]) / (mesh[i] - mesh[i-1])
        else:
            return (mesh[i+1] - x) / (mesh[i+1] - mesh[i])

def build_force_vector(f, mesh, n2=5):
    """
    Assemble the force (load) vector F where
         F[i] = ∫ f(x) * phi_i(x) dx,
    computed by integrating over the support of the i-th finite element function.
    
    Parameters:
      f     : The source function, f(x).
      mesh  : A list of node coordinates, e.g. [x0, x1, ..., xN].
      n2    : Number of Gauss-Legendre points.
      
    Returns:
      F     : A column vector (Sympy Matrix) of size (N+1) x 1.
    """
    num_nodes = len(mesh)  # = N+1
    F = sp.zeros(num_nodes, 1)
    
    # Loop over each finite element function phi_i.
    for i in range(num_nodes):
        total = 0.0
        
        # The support of phi_i:
        # For interior nodes, the support is [mesh[i-1], mesh[i+1]] split into two intervals.
        # For the left boundary, support is [mesh[0], mesh[1]].
        # For the right boundary, support is [mesh[-2], mesh[-1]].
        
        # Left subinterval (if it exists)
        if i > 0:
            x_left  = mesh[i-1]
            x_right = mesh[i]
            def integrand_left(x):
                return f(x) * phi_i(i, x, mesh)
            total += piecewise_GL(integrand_left, x_left, x_right, n2=n2)
        
        # Right subinterval (if it exists)
        if i < num_nodes - 1:
            x_left  = mesh[i]
            x_right = mesh[i+1]
            def integrand_right(x):
                return f(x) * phi_i(i, x, mesh)
            total += piecewise_GL(integrand_right, x_left, x_right, n2=n2)
        
        F[i, 0] = total
    return F

def f(s):
    return 0 
    


In [173]:
def extract_interior(S, F):
    S_int = S[1:-1, 1:-1]
    F_int = F[1:-1, :]

    return S_int, F_int

def c(S_int, F_int):
        c = S_int.LUsolve(F_int)
        return c


In [174]:
def approx_new(c, F):
    approx_new = c.T *F

    return approx_new[0,0]

def energy_norm(approximation):
    actual = 47/81
    energy_norm = actual - approximation
    return energy_norm

## Computing element error r(x)

In [175]:
def piecewise_linear_derivative(x_num, mesh, uh):
    """
    Return the piecewise-constant slope u_h'(x_num) for x_num in an element
    of the 1D mesh, or 0.0 if x_num is outside [mesh[0], mesh[-1]].
    """
    m = len(mesh) - 1
    if x_num < mesh[0] or x_num > mesh[m]:
        # outside the domain
        return 0.0
    
    # Find k so that mesh[k] <= x_num < mesh[k+1]
    # (or x_num == mesh[m], in which case we set k = m-1)
    k_found = None
    for k in range(m):
        if mesh[k] <= x_num < mesh[k+1]:
            k_found = k
            break
    if x_num == mesh[m]:
        k_found = m-1
    
    dx = mesh[k_found+1] - mesh[k_found]
    dy = uh[k_found+1] - uh[k_found]
    return dy / dx




In [176]:
# label each finite element for error calculation
def epsilon_k_sq():
    pass
     


def r(x_num, mesh, uh):
    # 'uh' and 'mesh' not strictly needed to compute a'(x), since a'(x)=0 except at 0.5.
    # The interior PDE says r(x) = f(x) + slope*a'(x). But a'(x)=0 almost everywhere,
    # so r(x) is just f(x).
    if 0 <= x_num < xd:
        return 2.0
    elif xd <= x_num <= 1:
        return 4.0


# def r(x_num, mesh, uh):
#     """
#     Residual r(x) = f(x) + u_h'(x)*a'(x), but with a'(x)=cos(x) if a(x)=2+sin(x).
#     => r(x) = f(x) + slope*cos(x_num).
#     """
#     slope = piecewise_linear_derivative(x_num, mesh, uh)
#     return f(x_num) + slope*math.cos(x_num)
# def element_residual_l2(x_left_element_i, x_right_element_i, mesh, uh):
#     """
#     Carry out Gauss-Legendre Quadrature of [r(x)]^2 on the element
#     [x_left_element_i, x_right_element_i], where
#       r(x) = f(x) + (u_h'(x))*cos(x).
#     We assume piecewise_linear_derivative + f are already defined.
#     """
#     def integrand_interior(x_val):
#         # We want [r(x_val)]^2
#         return (r(x_val, mesh, uh))**2
    
#     n2 = 5
#     xi, ci = gl.gauleg(n2)  # your routine that returns points & weights
#     # Integrate (r(x))^2 over this specific element
#     r_sq = GL(
#         xi=xi, ci=ci,
#         x_right=x_right_element_i,
#         x_left=x_left_element_i,
#         n2=n2,
#         func=integrand_interior
#     )
#     return r_sq

In [177]:

## use gauss legendre for each element. because it is discountinuous 

def piecewise_GL(integrand, x_left, x_right, n2):
    """
    Integrate 'integrand(x)' from x_left to x_right using Gauss-Legendre,
    but if [x_left, x_right] crosses x=1/3, split it into two parts:
      [x_left, 1/3]  and  [1/3, x_right]
    and sum the results.
    """
    discont =xd
    xi, ci = gl.gauleg(n2) 
    # If the entire interval is on one side of 1/3, do one normal G-L integral
    if x_right <= discont or x_left >= discont:
        return GL(xi =  xi, ci = ci, x_left=x_left, x_right=x_right, func=integrand, n2=n2)

    # Otherwise, we split at x=1/3
    # part1 = integral from x_left..1/3
    part1 = GL(xi =  xi, ci = ci, x_left=x_left, x_right=discont, func=integrand, n2=n2)
    # part2 = integral from 1/3..x_right
    part2 = GL(xi =  xi, ci = ci, x_left=discont, x_right=x_right, func=integrand, n2=n2)

    return part1 + part2


def element_residual_l2(x_left_element_i, x_right_element_i, mesh, uh):
    """
    Perform piecewise integration to handle the discontinuity at x=1/3.
    """
    def integrand_interior(x_val):
        return (r(x_val, mesh, uh))**2

    n2 = 5
    # Instead of one GL call, do:
    r_sq = piecewise_GL(
        integrand=integrand_interior,
        x_left=x_left_element_i,
        x_right=x_right_element_i,
        n2=n2
    )
    return r_sq


## Calculate boundary error 


In [178]:
def slope_at_node(mesh, uh, i, side):
    """
    Return the slope of the piecewise-linear solution u_h on the element
    *adjacent* to node i from the specified side ('left' or 'right').

    Parameters
    ----------
    mesh : list of floats, [x_0, x_1, ..., x_m]
           The node coordinates (strictly increasing).
    uh   : list of floats, [u_0, u_1, ..., u_m]
           The nodal values of u_h at mesh[i].
    i    : int
           The index of the node x_i in mesh.
    side : str
           Either 'left' or 'right'.
    
    Returns
    -------
    slope : float
        The slope in the element to the left or right of x_i.
        If x_i is at a boundary (e.g. i=0 and side='left'),
        we either raise an error or return 0.0, depending on your convention.
    """
    # Number of intervals is len(mesh)-1
    n = len(mesh) - 1

    if side == 'left':
        # The element to the left of node x_i is [x_{i-1}, x_i].
        # So we must have i >= 1 to have a left neighbor.
        if i == 0:
            slope = 0 
        else:
            dx = mesh[i] - mesh[i-1]
            dy = uh[i]   - uh[i-1]
            slope = dy/dx
        return slope
    
    elif side == 'right':
        # The element to the right of node x_i is [x_i, x_{i+1}].
        # So we must have i <= n-1 to have a right neighbor.
        if i >= n:
            slope = 0 
        else:
            dx = mesh[i+1] - mesh[i]
            dy = uh[i+1]  - uh[i]
            slope = dy/dx
        return slope

    else:
        raise ValueError("side must be 'left' or 'right'.")



In [179]:
# def j(mesh, uh, i):
#     if i == len(mesh)-1 or i == 0:
#         return 0.0
#     x_num = mesh[i+1]
#     slope_right = slope_at_node(mesh, uh, i+1, 'right')
#     sigma_right = 1 * a0(x_num) *slope_right
#     slope_left = slope_at_node(mesh, uh, i+1 , 'left')
#     sigma_left = -1 * a0(x_num) * slope_left 
#     j_x = sigma_right - sigma_left
#     return j_x


def j2(mesh, uh, i):
    """
    Compute the flux jump at node mesh[i+1] (the right boundary of element i),
    taking into account a jump in a0 at x=1/3.
    """
    # no jump at boundary or if i is last element
    if i == len(mesh)-1 or i == 0:
        return 0.0
    
    x_num = mesh[i]               # the node where we measure flux jump
    slope_right = slope_at_node(mesh, uh, i, 'right')
    slope_left  = slope_at_node(mesh, uh, i, 'left')
    
    discont = xd
    # Decide which "a0" to use for each side
    # if x_num < discont => both sides use a=1
    # if x_num > discont => both sides use a=2
    # if exactly x_num==discont => left side a=1, right side a=2
    if x_num < discont:
        a_left  = 1
        a_right = 1
    elif x_num > discont:
        a_left  = 2
        a_right = 2
    else:  # x_num == discont
        a_left  = 1
        a_right = 2
    
    sigma_right = a_right * slope_right
    sigma_left  = -1.0 * a_left * slope_left
    
    return sigma_right - sigma_left


In [180]:
def mesh_size(mesh, i):
    return mesh[i+1] - mesh[i]
    


def sum_of_error(i, mesh, nodal):
    x_left_element_i= mesh[i]
    x_right_element_i = mesh[i+1]
    h = x_right_element_i - x_left_element_i
    residual_sq = element_residual_l2(x_left_element_i, x_right_element_i, nodal, i)
    boundary_sq = j2(mesh, nodal, i=i) **2
    return h ** 2 * residual_sq + h * boundary_sq
    

def sum_of_error_list(mesh, nodal):
    error_list = []
    for i in list(range(len(mesh) -1)):
        error = sum_of_error(i = i, mesh = mesh, nodal = nodal)
        error_list.append(error)
    return error_list

def dorfler_marking(error_list, fraction=0.5):
    """
    Mark a minimal set of elements whose errors collectively
    exceed 'fraction' of the total error.

    Returns a list of element indices to refine.
    """
    # 1) sum of all errors
    error_list = np.asarray(error_list)
    total_err = sum(error_list)
    target = fraction * total_err

    # Binary search on the number of top elements needed (k)
    low, high = 1, len(error_list)
    best_k = None
    while low <= high:
        mid = (low + high) // 2
        # np.argpartition gets the indices of the mid largest errors (in arbitrary order)
        candidate = np.argpartition(error_list, -mid)[-mid:]
        candidate_sum = error_list[candidate].sum()
        
        if candidate_sum >= target:
            best_k = mid  # We have a candidate set of size mid
            high = mid - 1  # Try to see if a smaller set will work
        else:
            low = mid + 1

    # best_k now holds the minimal number of elements needed
    candidate = np.argpartition(error_list, -best_k)[-best_k:]
    # Optionally, sort these indices in descending order of error
    sorted_candidate = candidate[np.argsort(-error_list[candidate])]
    return sorted_candidate.tolist()



```
def adaptive_loop():
    # Initialize
    n_list, m_list, i_list = initial_data()
    for iteration in range(max_iterations):
        # 1) Solve
        current_C = solve_scF_once(n_list, m_list, i_list)
        
        # 2) Estimate error
        error_value = compute_error(current_C, n_list, m_list, i_list)
        
        # 3) Heap sort 
            if error_i > k * total error:
            add midpoint node to mesh index. 
        
        # 4) Mark for refinement if error too large
        for 
        if error_value < tolerance:
            print("Converged!")
            break
        
        # 5) Refine (e.g. split the mesh, update i_list, etc.)
        n_list, m_list, i_list = refine(n_list, m_list, i_list, current_C, error_value)

    return current_C

```


In [181]:
## heap sort of errors 
def element_selection_heap(errors, theta=0.8):
    """
    Dörfler-like marking using a heap approach.
    Returns the list of element indices with the largest errors 
    that collectively account for at least theta-fraction of total error.
    """
    import heapq

    # Pair up (error, index)
    err_pairs = [(err, i) for i, err in enumerate(errors)]
    # Sort in descending order of error
    sorted_desc = heapq.nlargest(len(err_pairs), err_pairs, key=lambda e: e[0])

    total_err = sum(e[0] for e in err_pairs)
    threshold = theta * total_err

    accum = 0.0
    to_refine = []
    for (err_val, idx) in sorted_desc:
        accum += err_val
        to_refine.append(idx)
        if accum >= threshold:
            break
    return to_refine


In [182]:
def solve_scF_once(n_list, m_list, mesh):
    """
    Build and solve the system S*C = F for a single iteration.

    Parameters
    ----------
    n_list, m_list : lists of indices (e.g. polynomial degrees)
    i_list         : indices for the piecewise-constant or piecewise-linear basis

    Returns
    -------
    C : Sympy Matrix
        The solution vector for the unknowns.
    """

    # Build the big Kronecker products
    matrixS1_Amn = A_S1(matrixA=A_matrix(n,m), matrixS1=S1_ji(func= a1, mesh = mesh))          # = kronecker(A, S1_mat)
    matrixS0_delta = S0_delta(matrixS0=S0_ji(func = a0, mesh = mesh), matrixdelta=delta(n,m))
    S = S_in_jm(matrixS1_Amn, matrixS0_delta) # S = matrixS0_delta + matrixS1_Amn

    # 2) Build the right-hand side vector F
    Fvec = build_force_vector(f, mesh, n2 =5)


    # 3) Extracting of interior nodes 
    S_int, F_int = extract_interior(S, Fvec)
    C_sol = c(S_int, F_int)  # calls your function C(...) which does: S_inv = S.inv(); C = S_inv*F

    return C_sol, Fvec, F_int


In [183]:
# criteria for refinement 
# initialize upper bound for termination 

epsilon = 0.1
def refine_mesh(mesh, element_index):
    """
    Refine the element [mesh[element_index], mesh[element_index+1]]
    by bisection. That is, insert a midpoint and return the new mesh.

    Example:
      mesh = [0, 0.5, 1], element_index=1 => the element [0.5, 1].
      We compute midpoint = 0.75, so new mesh = [0, 0.5, 0.75, 1].
    """
    x_left = mesh[element_index]
    x_right = mesh[element_index+1]
    midpoint = 0.5*(x_left + x_right)

    # Insert the midpoint after mesh[element_index]
    return mesh[:element_index+1] + [midpoint] + mesh[element_index+1:]

def element_selection(mesh, errors, epsilon):
    """
    Given a 1D mesh (list of node coords) and a list 'errors'
    of elementwise errors, refine any element whose error is > epsilon
    by bisection. Return the new mesh.
    """
    new_mesh = mesh[:]  # copy
    # We gather all elements to refine (descending order so indexing is stable).
    # Or just pick the largest error — depends on your marking strategy.
    elements_to_refine = []
    for elem_index, err_value in enumerate(errors):
        if err_value > epsilon:
            elements_to_refine.append(elem_index)

    # Sort in descending order so we refine from rightmost to leftmost,
    # avoiding index shifting.
    elements_to_refine.sort(reverse=True)

    return elements_to_refine

def element_refinement(mesh, element_index):
    new_mesh = mesh[:]
    for elem_idx in element_index:
        new_mesh = refine_mesh(new_mesh, elem_idx)

    return new_mesh

In [184]:
def assemble_nodal_values(C):
    """
    Given:
      C = [U_1, U_2, ..., U_{N-1}]  from the system solve
    Returns:
      U_full = [U_0, U_1, U_2, ..., U_{N-1}, U_N]  (with BCs)
    """
    N_minus_1 = len(C)          # number of interior unknowns
    U_full = [0]*(N_minus_1+2)  # +2 for endpoints
    U_full[0] = 0.0            # Dirichlet BC at x_0
    U_full[-1] = 0.0           # Dirichlet BC at x_N
    
    # Fill in interior:
    for i in range(N_minus_1):
        U_full[i+1] = C[i]
    
    return U_full


In [185]:
# def refinement_loop(epsilon):
#     """
#     1) Start with initial mesh
#     2) Solve once
#     3) Estimate errors
#     4) If all errors < epsilon, done. Else refine, go back to step 2.
#     """
#     mesh = [0, 0.5, 1]

#     # Keep track of (mesh, energy_norm) in each iteration
#     enorm_history = []
#     iteration_index = 0
#     while True:
#         # Solve for c_sol on the current mesh
#         c_sol, f_vec, f_int = solve_scF_once(n_list=n, m_list=m, mesh=mesh)

#         # Convert solution to nodal representation
#         nodal = assemble_nodal_values(c_sol)

#         # Estimate the elementwise errors
#         errors = sum_of_error_list(mesh=mesh, nodal=nodal)

#         # Mark which elements to refine
#         elements_to_refine = element_selection(mesh=mesh, errors=errors, epsilon=epsilon)
#         #elements_to_refine = dorfler_marking(errors, 0.2)


#         # For logging: compute the “approximation” and an “energy norm”
#         approximation = approx_new(c=c_sol, F=f_int)
#         enorm = energy_norm(approximation)

#         # Record in the history: current mesh + energy norm
#         enorm_history.append((iteration_index, enorm))

#         # If no elements exceed threshold => done
#         if not elements_to_refine:
#             break

#         # Otherwise refine
#         mesh = element_refinement(mesh, elements_to_refine)
#         iteration_index += 1
#         print("Refined mesh:", mesh)

#     # After loop, final solution is c_sol on final mesh
#     # Return everything, including the entire history
#     return mesh, c_sol, enorm_history


In [186]:
def refinement_loop2( epsilon):
    mesh = [0, 0.5, 1]

    # Keep track of (mesh, energy_norm) in each iteration
    enorm_history = []
    iteration_index = 0

    while True:
        S0 = S0_ji(func = a0, mesh = mesh)
        fvect = build_force_vector(f, mesh, 5)
        S0_int, f_int = extract_interior(S0, fvect)
        c_sol = c(S0_int, f_int)
        nodal = assemble_nodal_values(c_sol)
        errors = sum_of_error_list(mesh, nodal)
        elements_to_refine = element_selection(mesh=mesh, errors=errors, epsilon=epsilon)
        if not elements_to_refine:
            break

        # Otherwise refine
        mesh = element_refinement(mesh, elements_to_refine)
        iteration_index += 1
        print("Refined mesh:", mesh)
    return mesh, c_sol, enorm_history

In [188]:
import numpy as np
import matplotlib.pyplot as plt


mesh, c_sol, enorm_history = refinement_loop2(epsilon=0.1)


nodal = assemble_nodal_values(c_sol)  

x_nodal = np.array(mesh, dtype=float)
u_nodal = np.array(nodal, dtype=float)

# Define the exact solution
def exact_solution(x):
    if  0 <= x <= xd:
        return - x
    elif xd < x <= 1:
        return (0.5*x - 0.5)

# We'll plot the exact solution on a fine grid from 0..1
x_fine = np.linspace(0.0, 1.0, 200)
u_exact = exact_solution(x_fine)

plt.figure(figsize=(6,4))

# 1) Plot the piecewise-linear solution
plt.plot(
    x_nodal,
    u_nodal,
    marker='o',
    linestyle='-',
    color='blue',
    label='Numerical (Refined Mesh)'
)

# 2) Plot the exact solution as a smooth curve
plt.plot(
    x_fine,
    u_exact,
    color='red',
    linewidth=2,
    label='Exact: x(1-x)'
)

plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Refined Numerical Solution vs. Exact')
plt.grid(True)
plt.legend()
plt.show()


Refined mesh: [0, 0.25, 0.5, 0.75, 1]
Refined mesh: [0, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()